# AADS-ULoRA v5.5 - Google Colab Notebook

This notebook provides a complete setup for running the AADS-ULoRA v5.5 project on Google Colab with GPU support.

## Setup Instructions:
1. Upload this notebook to Google Colab
2. Change runtime type to GPU (Runtime → Change runtime type → GPU)
3. Run all cells sequentially
4. Mount your Google Drive to persist data between sessions

In [ ]:
# Check GPU availability
!nvidia-smi

import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU device: {torch.cuda.get_device_name(0)}")
    print(f"GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("⚠️  Warning: No GPU detected. Training will be very slow.")
    print("Go to Runtime → Change runtime type → Select GPU")
    print("\nRecommended: Select A100 GPU for best performance (if available)")
    print("A100 provides ~50GB GPU memory vs T4's ~15GB")
    print("If A100 not available, T4 can still work with smaller batch sizes")


In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

print("\nGoogle Drive mounted at /content/drive")
print("\nCreate a project directory in your Drive, e.g.:")
print("  /content/drive/MyDrive/aads-ulora-v5.5")


In [ ]:
# Clone the project from GitHub
import os
import shutil

# Define project directory
PROJECT_DIR = '/content/drive/MyDrive/bitirmeprojesi'
LOCAL_DIR = '/content/bitirmeprojesi'

# Clone from GitHub
print("Cloning AADS-ULoRA v5.5 from GitHub...")
!git clone https://github.com/EfeErim/bitirmeprojesi.git {LOCAL_DIR}

# Copy to Google Drive for persistence (optional)
if os.path.exists('/content/drive'):
    os.makedirs(PROJECT_DIR, exist_ok=True)
    shutil.copytree(LOCAL_DIR, PROJECT_DIR, dirs_exist_ok=True)
    print(f"✅ Project cloned to: {LOCAL_DIR}")
    print(f"✅ Project also saved to Drive: {PROJECT_DIR}")
    PROJECT_DIR = LOCAL_DIR  # Use local copy for faster access
else:
    PROJECT_DIR = LOCAL_DIR
    print(f"✅ Project cloned to: {LOCAL_DIR}")

print("\nProject structure:")
!ls -la {PROJECT_DIR}


In [ ]:
# Install system dependencies
!apt-get update -qq
!apt-get install -y -qq libgl1-mesa-glx libglib2.0-0 libsm6 libxext6 libxrender-dev

# Set project directory (from previous cell)
if 'PROJECT_DIR' not in locals():
    PROJECT_DIR = '/content/bitirmeprojesi'  # Default from clone

os.chdir(PROJECT_DIR)
print(f"Working in: {PROJECT_DIR}")

# Install Python dependencies
print("\nInstalling requirements...")
!pip install -q -r requirements.txt

# Install the package in development mode
print("\nInstalling AADS-ULoRA package...")
!pip install -q -e .

print("\n✅ Installation complete!")


In [ ]:
# Verify installation
import sys
print(f"Python version: {sys.version}")
print(f"Working directory: {os.getcwd()}")

# Check if src module is importable
try:
    import src
    print("✅ src module imported successfully")
except ImportError as e:
    print(f"❌ Failed to import src: {e}")

# List project structure
print("\nProject structure:")
!ls -la

# Check data directory
if os.path.exists('data'):
    print("\nData directory contents:")
    !ls -la data/
else:
    print("\n⚠️  No data directory found. You'll need to upload your dataset.")
    print("Upload data to: ./data/")
    print("Expected structure: data/tomato/, data/pepper/, data/corn/")


## Data Upload

You have several options to get your data into Colab:

### Option 1: Upload from local computer
Use the file browser on the left to upload your `data/` folder

### Option 2: Mount Google Drive and copy
If your data is already in Google Drive:
```python
shutil.copytree('/content/drive/MyDrive/path/to/data', './data', dirs_exist_ok=True)
```

### Option 3: Download from cloud storage
Use `!wget` or `!gdown` to download from URLs

### Expected data structure:
```
data/
├── tomato/
│   ├── phase1/
│   │   ├── healthy/
│   │   ├── early_blight/
│   │   └── ...
│   ├── val/
│   └── test/
├── pepper/
└── corn/
```

## Download Tomato Disease Dataset from Kaggle

This will download the [Tomato Disease Multiple Sources](https://www.kaggle.com/datasets/cookiefinder/tomato-disease-multiple-sources) dataset and organize it for training.

### Setup Kaggle API
1. Go to Kaggle → Account → API → Create API token
2. Upload `kaggle.json` to Colab when prompted
3. The dataset will be downloaded and automatically organized into the required structure

In [ ]:
# Install Kaggle API
!pip install -q kaggle

# Upload Kaggle API credentials
from google.colab import files
import os

print("Please upload your kaggle.json API credential file")
uploaded = files.upload()

if 'kaggle.json' in uploaded:
    # Move to .kaggle directory
    os.makedirs(os.path.expanduser('~/.kaggle'), exist_ok=True)
    shutil.move('kaggle.json', os.path.expanduser('~/.kaggle/kaggle.json'))
    os.chmod(os.path.expanduser('~/.kaggle/kaggle.json'), 0o600)
    print("✅ Kaggle API credentials configured")
else:
    print("⚠️  kaggle.json not found. Please upload it to continue.")
    print("Get it from: https://www.kaggle.com/account")


In [ ]:
# Download the tomato disease dataset from Kaggle
print("Downloading tomato disease dataset from Kaggle...")
print("This may take a few minutes depending on dataset size.")

# Create data directory
os.makedirs('./data', exist_ok=True)

# Download dataset
!kaggle datasets download -d cookiefinder/tomato-disease-multiple-sources -p ./data --unzip

print("\n✅ Dataset downloaded and extracted!")
print("\nChecking extracted files...")
!ls -la ./data/


In [ ]:
# Organize dataset into AADS-ULoRA expected structure
import random
from pathlib import Path

def organize_tomato_dataset(source_dir: str, target_dir: str):
    """
    Organize the Kaggle tomato dataset into AADS-ULoRA structure.
    
    Expected structure after organization:
    data/tomato/
    ├── phase1/  (training data)
    │   ├── healthy/
    │   ├── bacterial_spot/
    │   ├── early_blight/
    │   ├── late_blight/
    │   ├── leaf_curl/
    │   ├── leaf_mold/
    │   ├── septoria_leaf_spot/
    │   ├── spider_mites/
    │   ├── target_spot/
    │   ├── mosaic_virus/
    │   ├── yellow_leaf_curl_virus/
    │   └── ...
    ├── val/  (validation data)
    └── test/  (test data)
    """
    source = Path(source_dir)
    target = Path(target_dir)
    
    # Create target directories
    (target / 'tomato' / 'phase1').mkdir(parents=True, exist_ok=True)
    (target / 'tomato' / 'val').mkdir(parents=True, exist_ok=True)
    (target / 'tomato' / 'test').mkdir(parents=True, exist_ok=True)
    
    # The Kaggle dataset structure may vary. Let's detect it.
    print("Detecting dataset structure...")
    
    # Check common structures
    possible_structures = [
        source / 'train',
        source / 'valid',
        source / 'test',
        source / 'Train',
        source / 'Validation',
        source / 'Test'
    ]
    
    # Find the main data directories
    train_dir = None
    val_dir = None
    test_dir = None
    
    for p in possible_structures:
        if p.exists() and p.is_dir():
            if 'train' in p.name.lower():
                train_dir = p
            elif 'valid' in p.name.lower() or 'val' in p.name.lower():
                val_dir = p
            elif 'test' in p.name.lower():
                test_dir = p
    
    # If not found, check if source directly contains class folders
    if train_dir is None:
        # Assume the source directory itself contains class folders
        train_dir = source
        val_dir = None
        test_dir = None
    
    print(f"Detected structure:")
    print(f"  Train: {train_dir}")
    print(f"  Val: {val_dir}")
    print(f"  Test: {test_dir}")
    
    # Get class names from train directory
    class_folders = [d for d in train_dir.iterdir() if d.is_dir()]
    class_names = [d.name for d in class_folders]
    print(f"\nDetected classes: {class_names}")
    
    # Copy/ organize files
    tomato_target = target / 'tomato'
    
    # Organize training data
    print("\nOrganizing training data...")
    for class_name in class_names:
        source_class_dir = train_dir / class_name
        if not source_class_dir.exists():
            continue
            
        target_class_dir = tomato_target / 'phase1' / class_name
        target_class_dir.mkdir(parents=True, exist_ok=True)
        
        # Copy all images
        image_files = list(source_class_dir.glob('*.jpg')) + \
                     list(source_class_dir.glob('*.jpeg')) + \
                     list(source_class_dir.glob('*.png'))
        
        for img_file in image_files:
            shutil.copy2(img_file, target_class_dir / img_file.name)
        
        print(f"  {class_name}: {len(image_files)} images")
    
    # Organize validation data (if exists)
    if val_dir and val_dir.exists():
        print("\nOrganizing validation data...")
        for class_name in class_names:
            source_class_dir = val_dir / class_name
            if not source_class_dir.exists():
                continue
                
            target_class_dir = tomato_target / 'val' / class_name
            target_class_dir.mkdir(parents=True, exist_ok=True)
            
            image_files = list(source_class_dir.glob('*.jpg')) + \
                         list(source_class_dir.glob('*.jpeg')) + \
                         list(source_class_dir.glob('*.png'))
            
            for img_file in image_files:
                shutil.copy2(img_file, target_class_dir / img_file.name)
            
            print(f"  {class_name}: {len(image_files)} images")
    else:
        # Split training data into train/val (80/20)
        print("\nNo validation set found. Splitting training data (80/20)...")
        for class_name in class_names:
            source_class_dir = tomato_target / 'phase1' / class_name
            if not source_class_dir.exists():
                continue
                
            images = list(source_class_dir.glob('*.jpg')) + \
                    list(source_class_dir.glob('*.jpeg')) + \
                    list(source_class_dir.glob('*.png'))
            
            random.shuffle(images)
            split_idx = int(len(images) * 0.8)
            val_images = images[split_idx:]
            
            target_val_dir = tomato_target / 'val' / class_name
            target_val_dir.mkdir(parents=True, exist_ok=True)
            
            for img_file in val_images:
                shutil.move(str(img_file), str(target_val_dir / img_file.name))
            
            print(f"  {class_name}: moved {len(val_images)} images to validation")
    
    # Organize test data (if exists)
    if test_dir and test_dir.exists():
        print("\nOrganizing test data...")
        for class_name in class_names:
            source_class_dir = test_dir / class_name
            if not source_class_dir.exists():
                continue
                
            target_class_dir = tomato_target / 'test' / class_name
            target_class_dir.mkdir(parents=True, exist_ok=True)
            
            image_files = list(source_class_dir.glob('*.jpg')) + \
                         list(source_class_dir.glob('*.jpeg')) + \
                         list(source_class_dir.glob('*.png'))
            
            for img_file in image_files:
                shutil.copy2(img_file, target_class_dir / img_file.name)
            
            print(f"  {class_name}: {len(image_files)} images")
    else:
        # If no test set, use a portion of training as test
        print("\nNo test set found. Creating from training data (10%)...")
        for class_name in class_names:
            source_class_dir = tomato_target / 'phase1' / class_name
            if not source_class_dir.exists():
                continue
                
            images = list(source_class_dir.glob('*.jpg')) + \
                    list(source_class_dir.glob('*.jpeg')) + \
                    list(source_class_dir.glob('*.png'))
            
            if len(images) > 10:
                test_count = max(1, int(len(images) * 0.1))
                test_images = images[-test_count:]
                
                target_test_dir = tomato_target / 'test' / class_name
                target_test_dir.mkdir(parents=True, exist_ok=True)
                
                for img_file in test_images:
                    shutil.move(str(img_file), str(target_test_dir / img_file.name))
                
                print(f"  {class_name}: moved {len(test_images)} images to test")
    
    print("\n✅ Dataset organization complete!")
    print(f"\nFinal structure: {tomato_target}")
    print("\nClass distribution:")
    for split in ['phase1', 'val', 'test']:
        split_dir = tomato_target / split
        if split_dir.exists():
            print(f"  {split}:")
            for class_dir in sorted(split_dir.iterdir()):
                if class_dir.is_dir():
                    count = len(list(class_dir.glob('*.*')))
                    print(f"    {class_dir.name}: {count} images")

# Run organization
organize_tomato_dataset('./data', './data')

print("\n" + "="*50)
print("Dataset is ready for training!")
print("="*50)


## Training Pipeline

The AADS-ULoRA v5.5 system uses a three-phase training pipeline:

1. **Phase 1 - DoRA**: Base adapter training
2. **Phase 2 - SD-LoRA**: Continual Instance Learning
3. **Phase 3 - CONEC-LoRA**: Contrastive Learning for OOD detection

You can also train individual components as needed.

In [ ]:
# Train the Crop Router (L1)
print("Starting Crop Router training...")
!python -m src.router.simple_crop_router \
  --data_dir ./data \
  --crop tomato \
  --output_dir ./outputs/router \
  --epochs 10 \
  --batch_size 32 \
  --lr 1e-4 \
  --device cuda

print("\n✅ Router training complete!")


In [ ]:
# Phase 1: DoRA Training
print("Starting Phase 1 - DoRA training...")
print("A100 GPU: Using batch_size=32 for optimal performance")
!python -m src.training.phase1_training \
  --data_dir ./data \
  --crop tomato \
  --output_dir ./outputs/tomato_phase1 \
  --batch_size 32 \
  --epochs 50 \
  --lr 1e-4 \
  --device cuda

print("\n✅ Phase 1 complete!")


In [ ]:
# Phase 2: SD-LoRA Training
print("Starting Phase 2 - SD-LoRA training...")
print("A100 GPU: Using batch_size=32 for optimal performance")
!python -m src.training.phase2_sd_lora \
  --data_dir ./data \
  --crop tomato \
  --checkpoint ./outputs/tomato_phase1/checkpoints/best_model.pt \
  --output_dir ./outputs/tomato_phase2 \
  --batch_size 32 \
  --epochs 30 \
  --lr 5e-5 \
  --device cuda

print("\n✅ Phase 2 complete!")


In [ ]:
# Phase 3: CONEC-LoRA Training
print("Starting Phase 3 - CONEC-LoRA training...")
print("A100 GPU: Using batch_size=32 for optimal performance")
!python -m src.training.phase3_conec_lora \
  --data_dir ./data \
  --crop tomato \
  --checkpoint ./outputs/tomato_phase2/checkpoints/best_model.pt \
  --output_dir ./outputs/tomato_phase3 \
  --batch_size 32 \
  --epochs 20 \
  --lr 1e-5 \
  --device cuda

print("\n✅ Phase 3 complete!")
print("\n🎉 Full training pipeline complete!")


## Inference & Evaluation

Run inference on test data and evaluate model performance.

In [ ]:
# Evaluate the trained model
print("Running evaluation...")
!python -m src.evaluation.metrics \
  --data_dir ./data \
  --crop tomato \
  --checkpoint ./outputs/tomato_phase3/checkpoints/best_model.pt \
  --output_dir ./outputs/evaluation \
  --device cuda

print("\n✅ Evaluation complete!")


## Visualization

Generate visualizations of results, OOD detection, and training metrics.

In [ ]:
# Generate visualizations
print("Generating visualizations...")
!python -m src.visualization.visualization \
  --data_dir ./data \
  --crop tomato \
  --checkpoint ./outputs/tomato_phase3/checkpoints/best_model.pt \
  --output_dir ./outputs/visualizations \
  --device cuda

print("\n✅ Visualizations generated!")

# Display visualizations
from IPython.display import Image, display
import os

vis_dir = './outputs/visualizations'
if os.path.exists(vis_dir):
    print("\nGenerated plots:")
    for file in os.listdir(vis_dir):
        if file.endswith('.png') or file.endswith('.jpg'):
            print(f"  - {file}")
            display(Image(filename=os.path.join(vis_dir, file)))


## API Server

Start the FastAPI server for making predictions via REST API.

In [ ]:
# Start the API server in the background
!pip install -q uvicorn[standard]

print("Starting API server...")
print("Server will be available at: http://localhost:8000")
print("API documentation at: http://localhost:8000/docs")
print("\nPress the stop button to terminate the server\n")

!python -m uvicorn api.main:app --host 0.0.0.0 --port 8000 --reload


## Demo UI

Launch the Gradio demo interface for interactive testing.

In [ ]:
# Start the Gradio demo
!pip install -q gradio

print("Starting Gradio demo...")
print("Demo will be available at the URL shown below (usually with a public link)")
print("\nPress the stop button to terminate the demo\n")

!python demo/app.py


## Tips for Google Colab

1. **Session Timeout**: Colab sessions disconnect after ~12 hours or inactivity. Save checkpoints to Drive regularly.
2. **GPU Memory**: Colab provides ~15-16GB GPU memory (T4) or ~50GB (A100 if available). Use gradient checkpointing if needed.
3. **Storage**: Colab provides ~80GB temporary storage. Use Google Drive for persistent storage.
4. **Installation**: First run may take 10-15 minutes to install all dependencies.
5. **Data Transfer**: Upload large datasets to Google Drive first, then copy to Colab local storage for faster access.
6. **Checkpoints**: Always save model checkpoints to Google Drive to avoid losing progress.
7. **Free Tier Limits**: Colab has usage limits. Consider Colab Pro for longer sessions.

## Troubleshooting

- **CUDA out of memory**: Reduce batch size or use gradient accumulation
- **Import errors**: Ensure you ran `pip install -e .`
- **Data not found**: Verify your data directory structure
- **Slow training**: Make sure you're using GPU (check nvidia-smi output)

## Next Steps

1. Upload your dataset to `./data/`
2. Run data preparation
3. Train the crop router
4. Train each crop adapter through all 3 phases
5. Evaluate and visualize results
6. Deploy with the API or demo UI
